In [ ]:
import astropy.coordinates as coord
from astropy.io import ascii
from astropy.table import join, Table
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

I downloaded the 3 Bergemann tables from "Extended Data Tables" in [Bergemann et al. 2018](https://www.nature.com/articles/nature25490.pdf) by taking screenshots and using OCR to turn the screenshots into text. hashtag astronomy.

In [ ]:
tbl = None
for i in range(1, 3+1):
    filename = '../data/Bergemann{0}.txt'.format(i)
    
    if i == 1:
        tbl = ascii.read(filename)
    else:
        tbl = join(tbl, ascii.read(filename), keys='2MASS_ID')
        
tbl.write('../data/Bergemann2018.csv')

I downloaded the A13 master table from Ting's paper [Li et al. 2017](http://iopscience.iop.org/0004-637X/844/1/74/suppdata/apjaa7a0dt1_ascii.txt).

In [ ]:
tbl = ascii.read('../data/Li2017-A13.txt')

c = coord.SkyCoord(ra=coord.Angle(tbl['ra'], unit=u.hourangle),
                   dec=coord.Angle(tbl['dec'], unit=u.degree),
                   frame='fk5').transform_to(coord.ICRS)
tbl['ra'] = c.ra.degree
tbl['dec'] = c.dec.degree
tbl.write('../data/Li2017-A13.csv', overwrite=True)

I downloaded the TriAnd master table from Allyson's paper [Sheffield et al. 2014](http://iopscience.iop.org/article/10.1088/0004-637X/793/1/62/meta), removed all dwarf (D) stars, and removed the Triand1/2 distinction:

In [ ]:
tbl = ascii.read('../data/Sheffield2014-TriAnd.txt')

all_ra = []
all_dec = []
for row in tbl:
    _ra = row['2MASS_ID'][:7]
    _dec = row['2MASS_ID'][7:]
    ra = coord.Longitude(':'.join([_ra[:2], _ra[2:4], '.'.join([_ra[4:6], _ra[6]])]), 
                         unit=u.hourangle)
    dec = coord.Latitude(':'.join([_dec[:3], _dec[3:5], _dec[5:]]),
                         unit=u.degree)
    all_ra.append(ra)
    all_dec.append(dec)

c = coord.SkyCoord(ra=coord.Longitude(ra),
                   dec=coord.Latitude(dec),
                   frame='fk5').transform_to(coord.ICRS)
    
tbl['ra'] = c.ra.degree
tbl['dec'] = c.dec.degree
tbl.write('../data/Sheffield2014-TriAnd.csv', overwrite=True)

---

### Gaia cross-matches

In [ ]:
sheff = Table.read('../data/Sheffield2014-TriAnd.csv')
sheff.remove_columns(['ra', 'dec', 'l', 'b'])

sheff_gaia = Table.read('../data/Sheffield2014-xmatch-result.fits.gz')
sheff_gaia.rename_column('col2mass_id', '2MASS_ID')

sheff_master = join(sheff_gaia, sheff, keys='2MASS_ID')
sheff_master.write('../data/Sheffield2014-Gaia-master.fits')

In [ ]:
li = Table.read('../data/Li2017-A13.csv')
li.remove_columns(['ra', 'dec', 'l', 'b'])

li_gaia = Table.read('../data/Li2017-xmatch-result.fits.gz')

# Throw away A13-39 because it could be a binary:
li_gaia = li_gaia[li_gaia['name'] != 'A13-39']

li_master = join(li_gaia, li, keys='name')
li_master.write('../data/Li2017-Gaia-master.fits')